In [22]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, LeakyReLU, Flatten, Dense
from tensorflow.keras.models import Model
import numpy as np
import os
import cv2



In [23]:
# Constants
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 5  # Adjust based on your dataset
MAX_LABEL_LENGTH = NUM_CLASSES  # Ensure all labels have the same length

In [24]:
# Function to Load Images and Labels
def load_data(folder_path):
    image_dir = os.path.join(folder_path, "images")
    label_dir = os.path.join(folder_path, "labels")
    
    images, labels = [], []
    
    for img_name in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_name)
        label_path = os.path.join(label_dir, img_name.replace(".jpg", ".txt"))  # Assuming .txt labels
        
        # Load image
        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0  # Normalize
        images.append(img)
        
        # Load label
        label_array = np.zeros(MAX_LABEL_LENGTH)  # Default zero padding
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                label_data = f.read().strip().split()
                label_data = list(map(float, label_data))
                
                # Ensure the label fits within MAX_LABEL_LENGTH
                label_array[:min(len(label_data), MAX_LABEL_LENGTH)] = label_data[:MAX_LABEL_LENGTH]
                
        labels.append(label_array)  # Append fixed-length label
    
    return np.array(images), np.array(labels, dtype=np.float32)  # Ensure dtype consistency


In [25]:
# Load Dataset
X_train, y_train = load_data("Violence Detection.v1i.yolov8-obb/train")
X_valid, y_valid = load_data("Violence Detection.v1i.yolov8-obb/valid")


In [26]:
# Build YOLO-like Model
def build_yolo_model():
    input_layer = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    
    x = Conv2D(32, (3, 3), padding='same', activation=None)(input_layer)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    
    x = Conv2D(64, (3, 3), strides=2, padding='same', activation=None)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    
    x = Conv2D(128, (3, 3), strides=2, padding='same', activation=None)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    output_layer = Dense(NUM_CLASSES, activation='sigmoid')(x)  # Ensure correct output shape
    
    return Model(input_layer, output_layer)


In [27]:
# Build Model
model = build_yolo_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 401408)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │   102,760,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,856,133 (392.37 MB)

 Trainable params: 102,855,685 (392.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [28]:
# Train Model
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size=BATCH_SIZE)




Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - accuracy: 0.5570 - loss: -112.8556 - val_accuracy: 1.0000 - val_loss: -944.7106
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.5588 - loss: -1819.1399 - val_accuracy: 1.0000 - val_loss: -5259.7974
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.6510 - loss: -6370.5796 - val_accuracy: 1.0000 - val_loss: -10591.9854
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.6183 - loss: -18058.1270 - val_accuracy: 1.0000 - val_loss: -14332.3223
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.6541 - loss: -35310.2266 - val_accuracy: 1.0000 - val_loss: -39570.2383
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.6621 - loss: -69220.9375 - val_accuracy: 1.0000 - val_loss: -31994.1914
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.6710 - loss: -114898.0234 - val_accuracy: 1.0000 - val_loss: -233918.4375
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - a

In [29]:
# Save Model
model.save("violence_detection_model.h5")

In [30]:
def predict_image(image_path):
    # Load image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image.astype(np.float32) / 255.0  # Normalize
    
    # Add batch dimension
    image = np.expand_dims(image, axis=0)

    # Predict
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction)  # Get class index

    print(f"Prediction Probabilities: {prediction}")
    print(f"Predicted Class: {predicted_class}")

    return predicted_class


In [32]:
def predict_image(image_path):
    # Load image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image.astype(np.float32) / 255.0  # Normalize
    
    # Add batch dimension
    image = np.expand_dims(image, axis=0)

    # Predict
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction)  # Get class index

    print(f"Prediction Probabilities: {prediction}")
    print(f"Predicted Class: {predicted_class}")

    return predicted_class


In [33]:
import os
print(os.path.exists("Violence Detection.v1i.yolov8-obb/test/images/sample.jpg"))


False


In [34]:
predict_image(r"Violence Detection.v1i.yolov8-obb\test\images\bus_crash_91_png.rf.d8cb3fc8c2b8a900e11b4ed7dffe7269.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step
Prediction Probabilities: [[1. 1. 0. 0. 0.]]
Predicted Class: 0


0

In [35]:
predict_image("Violence Detection.v1i.yolov8-obb\\test\\images\\bus_crash_91_png.rf.d8cb3fc8c2b8a900e11b4ed7dffe7269.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Prediction Probabilities: [[1. 1. 0. 0. 0.]]
Predicted Class: 0


0

In [37]:
predict_image("Violence Detection.v1i.yolov8-obb\\train\\images\\269884206_3140306252913963_2359238654740430866_n_jpg.rf.02cbf117084c69658d807650d166323c.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Prediction Probabilities: [[1. 1. 0. 0. 0.]]
Predicted Class: 0


0

In [38]:
predict_image("static\\uploads\\fire2.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Prediction Probabilities: [[1. 1. 0. 0. 0.]]
Predicted Class: 0


0

In [39]:
predict_image("static\\uploads\\brooklyn_car_accidentpm.webp")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Prediction Probabilities: [[1. 1. 0. 0. 0.]]
Predicted Class: 0


0